# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np

import sqlite3
from sqlalchemy import create_engine
import os

import re
import nltk
import pickle

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.base import BaseEstimator, TransformerMixin

import warnings
warnings.simplefilter('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')

In [3]:
df = pd.read_sql_table('DisasterResponse',con = engine)
df.shape

(26028, 39)

In [4]:
df['message'].iloc[0]

'Weather update - a cold front from Cuba that could pass over Haiti'

In [5]:
df['original'].iloc[0]

'Un front froid se retrouve sur Cuba ce matin. Il pourrait traverser Haiti demain. Des averses de pluie isolee sont encore prevues sur notre region ce soi'

In [6]:
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)
Y.head(3)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,water,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    ''' Natural Language Processing: Normalize, Tokenize, Stem/Lemmatize
    '''
    
    # Convert text to lowercase and remove punctuation
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    # Tokenize words
    tokens = word_tokenize(text)
    
    # Remove Stop words, Stem & Lemmed words
    stop_word = stopwords.words("english")
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    
    #stemmed = [stemmer.stem(w) for w in tokens if w not in stop_word]
    lemmed = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_word]
    
    return lemmed

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

Multi target classification:
This strategy consists of fitting one classifier per target. This is a simple strategy for extending classifiers that do not natively support multi-target classification

In [8]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state = 0)

np.random.seed(0)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
Y_train_pred = pipeline.predict(X_train)
Y_test_pred = pipeline.predict(X_test)

In [11]:
# from sklearn.metrics import classification_report

col_names = Y.columns.tolist()
print(classification_report(Y_train, Y_train_pred, target_names= col_names))

                        precision    recall  f1-score   support

               related       0.99      0.99      0.99     13939
               request       1.00      0.92      0.96      3094
                 offer       1.00      0.69      0.81        83
           aid_related       1.00      0.97      0.98      7539
          medical_help       1.00      0.85      0.92      1416
      medical_products       1.00      0.84      0.91       881
     search_and_rescue       1.00      0.83      0.91       491
              security       0.99      0.77      0.87       320
              military       1.00      0.82      0.90       597
                 water       1.00      0.92      0.96      1135
                  food       1.00      0.96      0.98      2038
               shelter       1.00      0.92      0.96      1611
              clothing       1.00      0.82      0.90       278
                 money       1.00      0.81      0.89       435
        missing_people       0.99      

In [12]:
print(classification_report(Y_test, Y_test_pred, target_names= col_names))

                        precision    recall  f1-score   support

               related       0.84      0.92      0.88      5967
               request       0.83      0.45      0.58      1380
                 offer       0.00      0.00      0.00        35
           aid_related       0.76      0.60      0.67      3321
          medical_help       0.69      0.10      0.17       668
      medical_products       0.76      0.09      0.16       432
     search_and_rescue       0.68      0.06      0.12       233
              security       0.33      0.01      0.01       151
              military       0.75      0.05      0.09       263
                 water       0.87      0.26      0.40       537
                  food       0.80      0.56      0.66       885
               shelter       0.78      0.35      0.48       703
              clothing       0.79      0.12      0.21       127
                 money       0.38      0.03      0.05       169
        missing_people       0.00      

### 6. Improve your model
Use grid search to find better parameters. 

In [15]:
def performance_metric(y_true, y_pred):
    """Calculate median F1 score for all of the output classifiers
    Returns:
    score: float. Median F1 score for all of the output classifiers
    """
    f1_list = []
    for i in range(np.shape(y_pred)[1]):
        f1 = f1_score(np.array(y_true)[:, i], y_pred[:, i])
        f1_list.append(f1)
        
    score = np.median(f1_list)
    return score

In [16]:
parameters = {'vect__min_df': [5,10],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[10, 25], 
              'clf__estimator__min_samples_split':[2, 5, 10]}

scorer = make_scorer(performance_metric)
cv = GridSearchCV(pipeline, param_grid = parameters, scoring = scorer, verbose = 10)

# Find best parameters
np.random.seed(0)
tuned_model = cv.fit(X_train, Y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5, score=0.1385281385281385, total=  43.7s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   55.2s remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5, score=0.16143497757847533, total=  43.7s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.8min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=5, score=0.14468085106382977, total=  43.0s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=True, vect__min_df=10 


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  7.1min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=5, score=0.21408450704225354, total=  43.1s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=10 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  8.0min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=10, score=0.1845238095238095, total=  39.1s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=10 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=10, score=0.21750663129973472, total=  39.8s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=10 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, tfidf__use_idf=False, vect__min_df=10, score=0.22628951747088186, total=  39.0s
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=5 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=25, tfidf__use_idf=True, vect__min_df=5, score=0.2005899705014749, total= 1.4min
[CV] clf__estimator__min_samples_split=2

[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed: 81.5min finished


In [17]:
tuned_model.cv_results_

{'mean_fit_time': array([ 37.61458715,  34.29288705,  37.37542637,  33.69403235,
         79.54936131,  71.76845209,  79.3786033 ,  69.99145404,
         32.03317817,  29.93887281,  31.76294859,  29.365695  ,
         66.23909179,  62.38600246,  65.33809566,  60.0961473 ,
         30.49291142,  28.63490717,  29.59808421,  27.53292481,
         61.74419681,  57.21141521,  59.77261329,  54.99422868]),
 'std_fit_time': array([ 0.29438319,  0.24339276,  0.33642321,  0.33448901,  1.01911124,
         0.45252358,  0.24915408,  0.4156832 ,  0.31869422,  0.05530541,
         0.07114657,  0.09492371,  0.44695   ,  0.82088084,  0.55795519,
         0.27514058,  0.34530312,  0.25268047,  0.21482319,  0.11769937,
         0.17700611,  0.22159254,  0.18039763,  0.17811765]),
 'mean_score_time': array([ 5.85052617,  5.58566141,  5.62301071,  5.60202726,  7.45897738,
         7.21622229,  7.34170143,  7.04663809,  5.62263862,  5.50233587,
         5.70539935,  5.47167516,  7.33288272,  7.39038674,  7

In [18]:
# Best mean test score
np.max(tuned_model.cv_results_['mean_test_score'])

0.22979018117139663

In [19]:
# Parameters for best mean test score
tuned_model.best_params_

{'clf__estimator__min_samples_split': 10,
 'clf__estimator__n_estimators': 10,
 'tfidf__use_idf': True,
 'vect__min_df': 10}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
tuned_test_pred = tuned_model.predict(X_test)
print(classification_report(Y_test, tuned_test_pred, target_names= col_names))

                        precision    recall  f1-score   support

               related       0.85      0.93      0.89      5967
               request       0.79      0.50      0.61      1380
                 offer       0.00      0.00      0.00        35
           aid_related       0.73      0.72      0.72      3321
          medical_help       0.59      0.18      0.27       668
      medical_products       0.76      0.17      0.28       432
     search_and_rescue       0.67      0.15      0.25       233
              security       0.17      0.01      0.01       151
              military       0.60      0.14      0.23       263
                 water       0.86      0.45      0.59       537
                  food       0.80      0.66      0.72       885
               shelter       0.78      0.41      0.54       703
              clothing       0.65      0.16      0.25       127
                 money       0.67      0.08      0.15       169
        missing_people       1.00      

In [26]:
print(classification_report(Y_test, Y_test_pred, target_names= col_names))

             precision    recall  f1-score   support

          0       0.84      0.92      0.88      5967
          1       0.83      0.45      0.58      1380
          2       0.00      0.00      0.00        35
          3       0.76      0.60      0.67      3321
          4       0.69      0.10      0.17       668
          5       0.76      0.09      0.16       432
          6       0.68      0.06      0.12       233
          7       0.33      0.01      0.01       151
          8       0.75      0.05      0.09       263
          9       0.87      0.26      0.40       537
         10       0.80      0.56      0.66       885
         11       0.78      0.35      0.48       703
         12       0.79      0.12      0.21       127
         13       0.38      0.03      0.05       169
         14       0.00      0.00      0.00        97
         15       0.30      0.01      0.02       265
         16       0.81      0.17      0.28       357
         17       0.49      0.06      0.10   

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
def load_data(database_filepath):
    engine = create_engine('sqlite:///' + database_filepath)
    df = pd.read_sql_table ('DisasterResponse', con = engine)
    X = df['message']
    Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)
    col_names = Y.columns.tolist()
return X,Y,category_names

In [27]:
def build_model():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(estimator = AdaBoostClassifier()))
    ])
    
    parameters = {
        'vect__min_df':[1,10,50],
        'clf__estimator__learning_rate': [0.001, 0.01, 0.1],
        'tfidf__smooth_idf': [True, False]
    }
    
    model  = GridSearchCV(pipeline, param_grid = parameters, cv=2) 
    return model 

In [28]:
def evaluate_model(model, X_test, Y_test, category_names):
    Y_pred = model.predict(X_test)
    print(classification_report(Y_test, Y_pred, target_names=category_names, digits=2))

In [29]:
def save_model(model, model_filepath):
    with open(model_filepath, 'wb') as pkl_file:
        pickle.dump(model, pkl_file)
    pkl_file.close()

In [30]:
def main():
    X, Y, category_names = load_data(database_filepath)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3) 
    model = build_model()
    model.fit(X_train, Y_train)
    evaluate_model(model, X_test, Y_test, category_names)
    save_model(model, model_filepath)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
main()

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(tuned_model, open('disaster_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.